# IIC1005 2018-1: Ayudantía *Machine Learning*

###### Por: Daniela Flores y Antonio Ossa.

La idea de esta ayudantía es guiar el pre-procesamiento de datos que debes realizar para tu tarea. Además, veremos un poco de validación cruzada y entrenamiento de los modelos. El *dataset* que nos servirá para ilustrar gran parte de lo que podrías necesitar en tu tarea (¡la idea es que investigues tú también!) es **Qualitative Bankruptcy**.

## Qualitative Bankruptcy

Las columnas del dataset y sus valores posibles son:

```
1. Industrial Risk: {P,A,N}
2. Management Risk: {P,A,N}
3. Financial Flexibility: {P,A,N}
4. Credibility: {P,A,N}
5. Competitiveness: {P,A,N}
6. Operating Risk: {P,A,N}
7. Class: {B,NB}
8. Mysterious column: {P,A,N}
```

Para trabajar en la exploración y pre-procesamiento de datos usaremos nuestra querida librería `pandas`.

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv("qualitative-bankruptcy-data-ay.csv")
data.head()

### Observaciones

* Columna `id` no nos sirve.
* Nombres de columnas pocos descriptivos.
* Valores en filas no comparables.
* Explorar correlación entre columnas.

A continuación, nos haremos cargo de estas observaciones. En primer lugar, nos desharemos de la columna `id`.

In [ ]:
# Columna 'id' no nos sirve
data = data.drop('id', axis=1)

Ahora renombramos las columnas para saber bien a qué hacen referencia.

In [ ]:
# Nombres de columnas poco descriptivos
data = data.rename(columns={
    'IR': 'Industrial Risk',
    'MR': 'Management Risk',
    'FF': 'Financial Flexibility',
    'CR': 'Credibility',
    'CO': 'Competitiveness',
    'OP': 'Operating Risk',
    'MC': 'Mysterious Column'
})

Convertimos los valores *string* a datos numéricos que nos permitan trabajar sin problemas con los modelos en el futuro.

In [ ]:
# Valores en filas no comparables
meanings = {'P': 1, 'A': 0, 'N': -1}
data = data.applymap(lambda v: meanings.get(v, v))

Si dos o más columnas tienen una alta correlación, mantener ambas puede que no sea en extremo informativo. Para estudiar la correlacción entre columnas, `pandas` ofrece el método `corr` para sus `DataFrame`.

In [ ]:
# Explorar correlación entre columnas
data.corr()

In [ ]:
data.columns

Quizás si lo vemos graficado en una colorida matriz podamos identificar alguna rareza.

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
plt.matshow(data.corr())
plt.show()

Vemos que la correlación entre las columnas `Mysterious Column` y `Competitiveness` es 1. Esto se debe a que ¡sorpresa! ambas son la misma columna, solo tienen distinto nombre. Es importante notar que los datos fueron arreglados para que esto pasara. En la vida real, esto no es siempre tan evidente.

Procedemos a eliminar la columna `Mysterious Column`:

In [ ]:
data = data.drop('Mysterious Column', axis=1)

In [ ]:
data.head()

Podemos ver varias estadísticas interesantes gracias al método `describe` de los `DataFrame` de `pandas`.

In [ ]:
data.describe()

## Clasificación (simple)

¡Ahora viene un importante momento! Con el pre-procesamiento listo, procedemos a preparar los datos para entregarlos al modelo. Con esto en mente, utilizaremos `numpy` para separar los datos ($X$) del *target* ($y$, lo que queremos predecir). `ravel` nos permite obtener un arreglo unidimensional *aplanado*.

In [ ]:
from numpy import ravel

X = data[data.columns[:-1]]
y = data[['Class']]
y = ravel(y)

`sklearn` es la librería que contiene varias herramientas que hacen más sencillas las tareas relacionadas a aprendizaje de máquina. Para ejemplificar, utilizaremos los algoritmos de [*Support Vector Machine*](https://en.wikipedia.org/wiki/Support_vector_machine).

In [ ]:
from sklearn.svm import SVC

clf = SVC()
clf.fit(X, y)

Ahora veamos qué tan bien *predice* nuestro modelo.

In [ ]:
clf.score(X, y)

😱 Nuestro modelo predice perfectamente, qué maravilla. Lamentablemente, esto no es útil en la realidad. Si revisamos la documentación de [`sklearn`](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC.score), veremos que el método `score` debería recibir en $X$ observaciones de **prueba** y en $y$ las verdaderas etiquetas de esas observaciones de **prueba**. En nuestro ejemplo, **quisimos evaluar el modelo con los mismos datos con los que fue entrenado**, por esto era altamente probable que el *score* fuera perfecto, pues el modelo ya vio esas observaciones. Mostramos esto en ayudantía para que en tu tarea (y en el resto de tu vida) siempre tengas la precaución de **no entrenar con los datos de prueba**.

## *Cross validation*

Quizás te estés preguntando algo como: y si no puedo usar los mismos datos para entrenar y para testear, ¿qué hago para probar mi modelo? Ahí es cuando la validación cruzada entra en juego como una gran aliada. Con este *approach*, el set de entrenamiento se divide en $k$ conjuntos más pequeños. Lo siguiente se realiza para cada uno de estos $k$ conjuntos:
* Se entrena el modelo con $k-1$ sets.
* El modelo resultante se prueba con el set restante.

Así, procedemos a separar los datos en sets de entrenamiento y de *testing*. `sklearn` permite hacer esto con facilidad gracias a `train_test_split`. En nuestro ejemplo, dejaremos $60\%$ de los datos para entrenar y el $40\%$ restante para docimar.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import accuracy_score
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1)

A continuación, usaremos validación cruzada con 5 *folds*.

In [ ]:
cv = ShuffleSplit(n_splits=5, test_size=0.4, random_state=0)
scores = cross_val_score(clf, X_train, y_train, cv=cv)
scores

In [ ]:
clf = SVC()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

> En este caso, vemos que nuestra *accuracy* es bastante alta, lo que se debe a las características del *dataset*. Normalmente no será así :(

## Matriz de confusión

La matriz de confusión es una herramienta que permite visualizar el desempeño de nuestro modelo. En las columnas se ubica el número de predicciones de cada clase y en las filas están las instancias en la clase real.

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)